In [57]:
# Bibliotecas além do gerenciador Anaconda
!pip install spacy
!python -m spacy download pt_core_news_sm

!pip install wordcloud

!pip install gensim

!pip install tensorflow

You should consider upgrading via the 'c:\users\nakam\anaconda3\python.exe -m pip install --upgrade pip' command.


  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.2.0/pt_core_news_sm-3.2.0-py3-none-any.whl (22.2 MB)
[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


You should consider upgrading via the 'C:\Users\nakam\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\nakam\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\nakam\anaconda3\python.exe -m pip install --upgrade pip' command.


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


You should consider upgrading via the 'c:\users\nakam\anaconda3\python.exe -m pip install --upgrade pip' command.


  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=88b54f6b2d203d924b80881792d38d47ecb4d3814f706eb86587fd15ec4e0ea0
  Stored in directory: c:\users\nakam\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:
      Successfully uninstalled importlib-metadata-3.10.0


In [24]:
# Módulos básicos para manuseio de dados e arquivos
import numpy as np
import pandas as pd
from os.path import isfile, join
import nltk
import re
import string
import unicodedata

# Módulos para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

# Módulo para processamento de linguagem
import spacy

## Carregamento de textos

In [5]:
limited_news_path = r'Software\Fake.br-Corpus' #\fake_10 or \true_10
news_path = r'Software\Fake.br-Corpus\full_texts' #\fake or \true

paths = [limited_news_path, news_path]

In [28]:
def sortDir(dir_path: str, is_meta=False) ->list:
    '''
    Ordena os arquivos dentro de dir_path e os retorna no formato de lista.
    '''
    if is_meta:
        number_separator = "-meta.txt"
    else:
        number_separator = ".txt"

    first_list = os.listdir(dir_path)
    int_list = [int(element.split(number_separator)[0]) for element in first_list]
    int_list.sort()
    final_list = [(str(element) + number_separator) for element in int_list]

    return final_list

def txtToDataframe(path, is_limited=True):
    '''
    Function for converting full texts to a single DataFrame.
    '''
    if is_limited:
        true_files = [path+"\\true_10\\"+f for f in sortDir(dir_path = path+'\\true_10') if isfile(join(path+'\\true_10', f))]
        fake_files = [path+"\\fake_10\\"+f for f in sortDir(dir_path = path+'\\fake_10') if isfile(join(path+'\\fake_10', f))]
    else:
        true_files = [path+"\\true\\"+f for f in sortDir(dir_path = path+'\\true') if isfile(join(path+'\\true', f))]
        fake_files = [path+"\\fake\\"+f for f in sortDir(dir_path = path+'\\fake') if isfile(join(path+'\\fake', f))]
    
    texts = []
    labels = []
    
    for file in true_files:
        with open(file, encoding='utf-8-sig') as f:
            texts.append(f.read())
            labels.append('true')
    for file in fake_files:
        with open(file, encoding='utf-8-sig') as f:
            texts.append(f.read())
            labels.append('fake')
            
    df = pd.DataFrame(list(zip(texts,labels)),columns=['texts','labels'])
    
    # Com esta função, textos e labels foram inseridos em um DataFrame de maneira sequencial. Todas as notícias verdadeiras vêm
    # ANTES do bloco de notícias falsas.
    
    return df

def appendMetadata(path,df, is_limited=True):
    '''
    Function for appending metadata to previously generated news DataFrame.
    '''
    if is_limited:
        true_meta = [path+"\\true-meta-information-10\\"+f for f in sortDir(dir_path = path+'\\true-meta-information-10',is_meta=True) if isfile(join(path+'\\true-meta-information-10', f))]
        fake_meta = [path+"\\fake-meta-information-10\\"+f for f in sortDir(dir_path = path+'\\fake-meta-information-10',is_meta=True) if isfile(join(path+'\\fake-meta-information-10', f))]
    else:
        true_meta = [path+"\\true-meta-information\\"+f for f in sortDir(dir_path = path+'\\true-meta-information',is_meta=True) if isfile(join(path+'\\true-meta-information', f))]
        fake_meta = [path+"\\fake-meta-information\\"+f for f in sortDir(dir_path = path+'\\fake-meta-information',is_meta=True) if isfile(join(path+'\\fake-meta-information', f))]
    

    #true_meta e fake_meta são listas com todas os paths para arquivos de metadata.
    
    columns = ["author", "source", "category", "date","tokens","words_without_punctuation","types","number_of_links","uppercase_words","verbs","subjuntive_imperative","nouns","adjectives","adverbs","modal_verbs","singular_first_and_second_personal_pronouns","plural_first_personal_pronouns","pronouns","pausality","characters","avg_sentence_length","avg_word_length","percentage_of_spelling_errors","emotiveness","diversity"]
    
    true_metadata = pd.DataFrame(columns=columns)
    fake_metadata = pd.DataFrame(columns=columns)
    
    for file in true_meta:
        #print(file)
        aux = pd.read_csv(file, header=None, sep = '\n').transpose()
        aux.columns = columns
        true_metadata=true_metadata.append(aux)
        
        
    for file in fake_meta:
        #print(file)
        aux = pd.read_csv(file, header=None, sep = '\n').transpose()
        aux.columns = columns
        fake_metadata=fake_metadata.append(aux)
        
    
    metadata = pd.DataFrame(columns=columns)
    metadata = metadata.append(true_metadata,ignore_index=True)
    metadata = metadata.append(fake_metadata,ignore_index=True) 


    complete_df = pd.concat([df,metadata],axis=1)
    # Este DataFrame possui todos os textos/labels (2 colunas) e metadata (25 colunas).
    
    return complete_df

In [29]:
ai = int(input('''0 - Base com 10 notícias verdadeiras e 10 notícias falsas
1 - Base completa de notícias
'''))

path = paths[ai]

if ai == 0:
    data = txtToDataframe(path) # Dataframe contendo notícias e labels.
    complete_data = appendMetadata(path,data) # Dataframe contendo notícias, labels e metadata.
else:
    data = txtToDataframe(path,is_limited=False)
    complete_data = appendMetadata(path,data,is_limited=False)

0 - Base com 10 notícias verdadeiras e 10 notícias falsas
1 - Base completa de notícias
0


In [12]:
complete_data.columns

Index(['texts', 'labels', 'author', 'source', 'category', 'date', 'tokens',
       'words_without_punctuation', 'types', 'number_of_links',
       'uppercase_words', 'verbs', 'subjuntive_imperative', 'nouns',
       'adjectives', 'adverbs', 'modal_verbs',
       'singular_first_and_second_personal_pronouns',
       'plural_first_personal_pronouns', 'pronouns', 'pausality', 'characters',
       'avg_sentence_length', 'avg_word_length',
       'percentage_of_spelling_errors', 'emotiveness', 'diversity'],
      dtype='object')

In [30]:
complete_data['texts'][0]

'O Podemos decidiu  expulsar o deputado federal Carlos Gaguim do partido após a Polícia Federal fazer buscas a apreensões no gabinete dele na Câmara. Com isso, a legenda abre espaço para receber a senadora expulsa pelo PMDB, Katia Abreu. Por meio de nota, a legenda informou que o afastamento do parlamentar já era algo acordado entre os filiados da sigla.  “Ainda que o parlamentar tenha comunicado a conclusão de sua desfiliação para esta semana, diante dos fatos noticiados hoje, a Executiva Nacional do Podemos solicita o imediato cancelamento de sua filiação dos quadros do partido”.\n\nO partido, que no passado chegou a cogitar lançar o parlamentar como candidato ao Senado, diz que “apoia a investigação com a ampla apuração dos eventuais crimes cometidos e a consequente responsabilização dos envolvidos, para que todos sejam punidos com o máximo rigor da Lei, independentemente de posição ou cargo ocupado”. '

## Preprocessamento de textos

In [15]:
# Carregando o pacote de língua portuguesa para o processador Spacy
nlp = spacy.load('pt_core_news_sm')

In [42]:
# Defininido funções de preprocessamento

def removePunct(text):
    '''
    Removes any punctuation included in string.punctuation.
    '''
    translator = text.maketrans({key:'' for key in string.punctuation+'“”'}) # Translates any punctuation into ''
    return text.translate(translator)

def removeNumbers(text):
    '''
    Removes any number character in text.
    '''
    return re.sub('[0-9]', '' , text) # Translates any number into ''

def removeStopWords(string):
    '''
    Removes any portuguese stopwords, using Spacy's standard package.
    '''
    doc = nlp(string)
    return ' '.join([token.text for token in doc if token.is_stop is False])

def lemmatize(string):
    '''
    Lemmatizes text word-by-word. Notice that lemmatizing is not as harsh as stemming, which makes the final text easier to read and understand in common language.
    '''
    doc = nlp(string)
    return ' '.join([token.lemma_ for token in doc])

def prep(string, useStopWords = True, lemma = True):
    '''
    Executes previously defined preprocessing in text.
    '''

    result = removeNumbers(removePunct(string)).lower()
    
    if useStopWords and lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc if token.is_stop is False])
    elif useStopWords:
        doc = nlp(result)
        result = ' '.join([token.text for token in doc if token.is_stop is False])
    elif lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc])
    return result.replace('\n',"")

In [43]:
# Realizando preprocessamento de textos presentes no Dataframe de notícias completo.

complete_data['texts'] = complete_data['texts'].apply(prep)

In [44]:
complete_data['texts'][0]

'decidir       expulsar deputar federal carlos gaguim polícia federal buscar apreensão gabinete dele câmara legendar abrir espaçar receber senador expulsar pmdb katia abreu notar legendar informar afastamento parlamentar acordar filiar sigla        parlamentar comunicar conclusão desfiliação semana fato noticiar hoje executivo nacional solicitar imediato cancelamento filiação quadrar       passar chegar cogitar lançar parlamentar candidatar senado   investigação amplo apuração crime cometer consequente responsabilização envolver punir rigor lei independentemente cargo ocupar'

In [48]:
# Assignando variáveis dependentes e independentes

y = complete_data['labels'].values # y is strings for labels; but should be fake-0/true-1
X = [d.split() for d in complete_data['texts'].tolist()] # X is a list of lists of words.

In [52]:
print(X[0])

['decidir', 'expulsar', 'deputar', 'federal', 'carlos', 'gaguim', 'polícia', 'federal', 'buscar', 'apreensão', 'gabinete', 'dele', 'câmara', 'legendar', 'abrir', 'espaçar', 'receber', 'senador', 'expulsar', 'pmdb', 'katia', 'abreu', 'notar', 'legendar', 'informar', 'afastamento', 'parlamentar', 'acordar', 'filiar', 'sigla', 'parlamentar', 'comunicar', 'conclusão', 'desfiliação', 'semana', 'fato', 'noticiar', 'hoje', 'executivo', 'nacional', 'solicitar', 'imediato', 'cancelamento', 'filiação', 'quadrar', 'passar', 'chegar', 'cogitar', 'lançar', 'parlamentar', 'candidatar', 'senado', 'investigação', 'amplo', 'apuração', 'crime', 'cometer', 'consequente', 'responsabilização', 'envolver', 'punir', 'rigor', 'lei', 'independentemente', 'cargo', 'ocupar']


## Tokenization (TensorFlow)

In [58]:
# Importando módulos para tokenização de textos
import gensim

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [59]:
# Realizando slicing de textos

test_limit = 15

training_sentences = X[0:test_limit]
testing_sentences = X[test_limit:]

training_labels = y[0:test_limit]
testing_labels = y[test_limit:]

In [60]:
# Gerando dicionário de tokens (com base nos textos de treinamento)

tokenizer = Tokenizer(oov_token='<OOV>')

tokenizer.fit_on_texts(training_sentences)

In [ ]:
# Realizando a sequencialização das bases de treinamento e teste

training_sequences = tokenizer.texts_to_sequences(training_sentences)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

In [ ]:
# Análise da quantidade de notícias maiores do que max_length

max_length = 2000

#plt.hist([len(x) for x in X], bins = 750)
#plt.show()

nos = np.array([len(x) for x in X])
print("There are "+ str(len(nos[nos>max_length])) + " news longer than "+ str(max_length) + " words.")

In [ ]:
# Realização da truncagem (padding) das sequências tokenizadas

training_padded = pad_sequences(training_sequences,maxlen=max_length,padding='post',truncating='pre')
testing_padded = pad_sequences(testing_sequences,maxlen=max_length,padding='post',truncating='pre')

In [ ]:
vocab = tokenizer.word_index
inv_vocab = {v: k for k, v in vocab.items()} # reconverte o token para a palavra correpondente.

training_texts = []
testing_texts = []

# Com estas listas "_texts", temos os textos novamente completos, mas agora com 'OOV's para palavras desconhecidas
# na base de testes.
for sequence in training_sequences:
    news=[]
    for token in sequence:
        news.append(inv_vocab[token])  #inv_vocab[token] é a palavra correspondente ao token.
    training_texts.append(news)


for sequence in testing_sequences:
    news=[]
    for token in sequence:
        news.append(inv_vocab[token])
    testing_texts.append(news)

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score